<a href="https://colab.research.google.com/github/marianaalmeid/mne-python/blob/main/Data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data preprocessing**

In [ ]:
pip install mne

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import mne
from mne import Epochs, pick_types, events_from_annotations, create_info, find_events, read_annotations, pick_events, read_events, read_epochs_eeglab, read_evokeds
from mne.channels import read_layout, read_custom_montage, find_layout, make_standard_montage, make_eeg_layout, make_dig_montage
from mne.io import concatenate_raws, read_raw_gdf, read_raw_eeglab
from mne.time_frequency import (tfr_multitaper, tfr_stockwell, tfr_morlet,tfr_array_morlet)
from mne.preprocessing import ICA
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename
import warnings
from PIL import Image
warnings.filterwarnings('ignore')

In [ ]:
file = '/NF4_online_gdf_2_1_2019.07.16_11.43.59.gdf' #PLEASE UPLOAD SAMPLE FILES
raw = read_raw_gdf(file, preload=True) 

Extracting EDF parameters from /NF4_online_gdf_2_1_2019.07.16_11.43.59.gdf...
GDF file detected


IndexError: ignored

**This cell reads the location file, applies the correct name to each channel and shows a aurface map of each electrode location.**

In [ ]:
ch_names = raw.ch_names
montage = read_custom_montage('/CLACS-32_REFinTP9_NF Teresa.bvef') #PLEASE UPLOAD SAMPLE FILES
montage.plot(kind='topomap')
ch_loc = montage.ch_names
c = dict(zip(ch_names, ch_loc)) #combine names with locations
raw.rename_channels(c)
raw.set_montage(montage)
print('Number of channels marked as bad:', len(raw.info['bads']))

# Apply band-pass filter (1-45 Hz) and re-reference to average

In [ ]:
lo_freq = 1.
hi_freq = 45.
filtered = raw.filter(lo_freq, hi_freq, fir_design='firwin', skip_by_annotation='edge')
filtered.set_eeg_reference(ref_channels='average', projection = True).apply_proj() 

# Epoching into sets and trials (C)
This cell works for *Cristiano'*s data. Takes each annotations present in the raw files and croppes them into epochs containing either sets or trials.



> SETS


In [ ]:
sets_id = {'1':1,'7':37,'13':8,'19':14,'25':21}
events_sets, _ = events_from_annotations(raw, event_id=sets_id)
tmin_1=-0.5 
tmax_1=360
cropped_sets = filtered.copy()
picks_sets = pick_types(cropped_sets.info, meg=False, eeg=True, stim=False, 
                        eog=False,exclude='bads')
epochs_sets = Epochs(cropped_sets, events_sets, 
                     sets_id , tmin_1, tmax_1, 
                     proj=True,picks=picks_sets, preload=True,baseline=None)



> TRIALS



In [ ]:
trials_id={'1': 1, '10': 2, '11': 6, '12': 7, '13': 8, 
 '14': 9, '15': 10, '16': 11, '17': 12, '18': 13, '19': 14, '2': 15, 
 '20': 16, '21': 17, '22': 18, '23': 19, '24': 20, '25': 21, '26': 22,
 '27': 23, '28': 24, '29': 25, '3': 26, '30': 27, '4': 31, '5': 32,'6': 36,
 '7': 37, '8': 38, '9': 39}
ts_trials, _ = events_from_annotations(raw, event_id=trials_id)
tmin_2=-0.5
tmax_2=60
cropped_trials = filtered.copy()
picks_trials = pick_types(cropped_trials.info, meg=False, eeg=True, stim=False, 
                        eog=False, exclude='bads')
epochs_trials = Epochs(cropped_trials, events_trials, trials_id, tmin_2,
                       tmax_2, proj=True,picks=picks_trials, preload=True,
                       baseline=None)

# Epoching into sets and trials (T)
This cell works for *Teresa'*s data. Takes each annotations present in the raw files and croppes them into epochs containing either sets or trials.



> SETS


In [ ]:
sets_id = {'1':1,'7':37,'13':8,'19':14,'25':21}
events_sets, _ = events_from_annotations(raw, event_id=sets_id)
tmin_1=-0.5 
tmax_1=360
cropped_sets = filtered.copy()
picks_sets = pick_types(cropped_sets.info, meg=False, eeg=True, stim=False, 
                        eog=False,exclude='bads')
epochs_sets = Epochs(cropped_sets, events_sets, sets_id , tmin_1, tmax_1, 
                     proj=True,picks=picks_sets, preload=True,baseline=None)




> TRIALS



In [ ]:
trials_id={'33282':6}
events_trials, _ = events_from_annotations(raw, event_id=trials_id)
tmin_2=-0.5
tmax_2=60
cropped_trials = filtered.copy()
picks_trials = pick_types(cropped_trials.info, meg=False, eeg=True, stim=False, 
                          eog=False, exclude='bads')
epochs_trials = Epochs(cropped_trials, events_trials, trials_id, tmin_2,
                       tmax_2, proj=True,picks=picks_trials, preload=True,
                       baseline=None)

Plotting channels

In [ ]:

'''
channel = input("Choose a channel number to visualize:")
#epochs_sets.plot()
epochs_sets.plot_image(int(channel), cmap='interactive', show=True) 
epochs_sets.plot_image(combine='gfp', sigma=2., cmap="YlGnBu_r")
epochs_sets.plot_topo_image(vmin=-250, vmax=250, title='ERF images', sigma=2.)

# %%
# SEE TOPOPLOTS HEATMAP ON THE SCALP
# more plotting functions, divde different eeg bands
# also look at ICA
# apply fft or plot frequency domain 
# MAKE IT WORK FOR ANY FILE, extract annotations timing

epochs_sets.plot_psd(fmin=1., fmax=40., average=True) # Channel types by averaging across epochs


# %%

#fig, axis = plt.subplots(1, 2, figsize=(7, 4))
#epochs_sets.plot_psd_topomap(ch_type='eeg', normalize=False)
#epochs_sets.plot_psd_topomap(ch_type='eeg', normalize=True,show=False)#,axes=axis[1])
#epochs_trials.plot_psd_topomap(ch_type='eeg', normalize=False)
'''


# Applying ICA for noise removal

In [ ]:

#RUN ICA on Raw cropped
ica_raw = ICA(n_components=19,method='fastica').fit(cropped_sets) #Applies ICA to the epoched data
ica_raw.plot_sources(cropped_sets) # Shows the time series of the ICs (SHOW ONLY 1st SET)
components = ica_raw.plot_components(cmap='jet') # plot components
components.save(cur_dir, 'JPEG')

#VISUALIZING ICA AND CHOOSE AT LEAST ONE COMPONENT
see = int(input("Choose ICA components to see:"))
ica_raw.plot_properties(cropped_sets,picks=see)
ica_raw.plot_overlay(cropped_sets, exclude=[see], picks='eeg')

# Removing components based on plots and comparison
#lista = []
#remove = [int(input("Choose ICA components to remove:"))]
ica_raw.exclude = [0,1]
reconst_epochs = cropped_sets.copy()
ica_raw.apply(reconst_epochs)
reconst_epochs.plot()

Generating new Epochs (after ICA)

In [ ]:
new_epochs_sets = Epochs(reconst_epochs, events_sets, sets_id, tmin_1, tmax_1, proj=True,picks=picks_sets, preload=True,baseline=None)

# Comparing topographic maps

In [ ]:
epochs_sets.plot_psd(fmin=1., fmax=40., average=True)
epochs_sets.plot_psd_topomap(ch_type=()='eeg',cmap='jet', normalize=False)
new_epochs_sets.plot_psd(fmin=1., fmax=40., average=True) # Channel types by averaging across epochs
new_epochs_sets.plot_psd_topomap(ch_type='eeg',cmap='jet', normalize=False)

Saves new data

In [ ]:
new_epochs_sets.save('epochs_S1')
reconst_epochs.save('raw_S1')